## Part 1: Preprocessing

In [319]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [320]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [321]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']].copy()
y_df

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [322]:
# Create a list of at least 10 column names to use as X data
X_data = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_data].copy()

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [323]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

In [324]:
X_train
X_train.OverTime.value_counts()

OverTime
No     780
Yes    322
Name: count, dtype: int64

In [325]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder instance
ohe = OneHotEncoder(sparse_output=False)  # Ensure the output is a dense array
# Fit the "OverTime column"
ohe.fit(X_train[['OverTime']])
X_train_ohe = ohe.transform(X_train[['OverTime']])

X_train_numeric = X_train.copy()
X_train_numeric["OverTime"] = X_train_ohe[:, 1]
print(X_train_numeric.OverTime.value_counts())

# Repeat for the test data
X_test_ohe = ohe.transform(X_test[['OverTime']])
X_test_numeric = X_test.copy()
X_test_numeric["OverTime"] = X_test_ohe[:, 1]

OverTime
0.0    780
1.0    322
Name: count, dtype: int64


In [326]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train_numeric)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train_numeric)
X_test_scaled = scaler.transform(X_test_numeric)

In [327]:
y_train.Department.value_counts()

Department
Research & Development    721
Sales                     336
Human Resources            45
Name: count, dtype: int64

In [328]:
# Create a OneHotEncoder for the Department column
enc_dpt = OneHotEncoder(handle_unknown='ignore')

# Fit the encoder to the training data
enc_dpt.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_encoded_train = enc_dpt.transform(y_train[['Department']]).toarray()
print(department_encoded_train)
y_train[['Research & Development', 'Sales', 'Human Resources']] = department_encoded_train
# Delete the original OverTime column from X_train
y_train.drop(columns=['Department'], inplace=True)

department_encoded_test = enc_dpt.transform(y_test[['Department']]).toarray()
y_test[['Research & Development', 'Sales', 'Human Resources']] = department_encoded_test
y_test.drop(columns=['Department'], inplace=True)

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [329]:
y_train.Attrition.value_counts()

Attrition
No     913
Yes    189
Name: count, dtype: int64

In [330]:
# Create a OneHotEncoder for the Attrition column
enc_attr = OneHotEncoder(handle_unknown='ignore')

# Fit the encoder to the training data
enc_attr.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_encoded_train = enc_attr.transform(y_train[['Attrition']]).toarray()
print(attrition_encoded_train)
y_train[['Attrition_No', 'Attrition_Yes']] = attrition_encoded_train
# Delete the original OverTime column from X_train
y_train.drop(columns=['Attrition'], inplace=True)

attrition_encoded_test = enc_attr.transform(y_test[['Attrition']]).toarray()
y_test[['Attrition_No', 'Attrition_Yes']] = attrition_encoded_test
y_test.drop(columns=['Attrition'], inplace=True)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


## Part 2: Create, Compile, and Train the Model

In [331]:
# Find the number of columns in the X training data.
print(X_train_scaled.shape[1])

# Create the input layer
input_layer = layers.Input(shape=(X_train_scaled.shape[1],), name='input_layer')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu')(input_layer)  # Adjusted kernel size
shared2 = layers.Dense(128, activation='relu')(shared1)  # Adjusted pooling size


10


In [332]:
y_train

,Research & Development,Sales,Human Resources,Attrition_No,Attrition_Yes
1343,0.0,1.0,0.0,1.0,0.0
1121,0.0,0.0,1.0,1.0,0.0
1048,0.0,0.0,1.0,1.0,0.0
1393,0.0,0.0,1.0,1.0,0.0
527,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...
1130,0.0,1.0,0.0,1.0,0.0
1294,0.0,1.0,0.0,1.0,0.0
860,0.0,1.0,0.0,0.0,1.0
1459,0.0,1.0,0.0,1.0,0.0


In [333]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu')(shared2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)

In [334]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(64, activation='relu')(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [335]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 128)       │      8,320 │ dense_16[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 32)        │      4,128 │ dense_17[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 64)        │      8,256 │ dense_17[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_18[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │        130 │ dense_19[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 21,637 (84.52 KB)

 Trainable params: 21,637 (84.52 KB)

 Non-trainable params: 0 (0.00 B)

In [336]:
# Train the model
history = model.fit(
    X_train_scaled,
    {'department_output': department_encoded_train, 'attrition_output': attrition_encoded_train},
    epochs=100,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - attrition_output_accuracy: 0.7672 - attrition_output_loss: 0.5951 - department_output_accuracy: 0.5402 - department_output_loss: 0.9636 - loss: 1.5588 - val_attrition_output_accuracy: 0.7873 - val_attrition_output_loss: 0.5294 - val_department_output_accuracy: 0.6063 - val_department_output_loss: 0.8620 - val_loss: 1.3940
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8383 - attrition_output_loss: 0.4396 - department_output_accuracy: 0.6788 - department_output_loss: 0.7485 - loss: 1.1880 - val_attrition_output_accuracy: 0.7873 - val_attrition_output_loss: 0.4963 - val_department_output_accuracy: 0.6063 - val_department_output_loss: 0.8337 - val_loss: 1.3327
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8313 - attrition_output_loss: 0.4165 - department_output_accuracy: 0.6837 - department_output_loss: 0.7400 - loss: 1.1568 - val_attrition_output_accuracy: 0.7873 - 

In [337]:
# Evaluate the model with the testing data
model.evaluate(
    X_test_scaled,
    {'department_output': department_encoded_test, 'attrition_output': attrition_encoded_test}
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8117 - attrition_output_loss: 1.5926 - department_output_accuracy: 0.5448 - department_output_loss: 3.0271 - loss: 4.6307 


[4.215399265289307,
 2.825141191482544,
 1.3183854818344116,
 0.83423912525177,
 0.551630437374115]

In [338]:
# Print the accuracy for both department and attrition
print("Attrition predictions accuracy: ", history.history['attrition_output_accuracy'][-1])
print("Department predictions accuracy: ", history.history['department_output_accuracy'][-1])

Attrition predictions accuracy:  1.0
Department predictions accuracy:  1.0


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Because the attrition and department columns are not balanced, accuracy would not be the best metric. The attrition data has a much higher number of no's than yes's. Department has much less data for human resources than research and development
2. I used softmax for department because there are three possible outputs, so it is multi-class classification. I used sigmoid for attrition because it is binary classification, yes or no.
3. I could try dropout to prevent/reduce overfitting. I could try adjusting some of the model parameters and/or try hyper parameter tuning. I could also try adding more layers and varying the output shape of each layer. Any of these methods could improve performance.